## Cobb County right angle crash data feature engineering
### Import the modules and libraries.

In [1]:
import os
import arcpy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import astral
from astral import LocationInfo
from astral.sun import sun
%matplotlib inline


### Preprocess data
Add several fields and do the spatial joins to add aadt and intersection to the crash data.
Calculate curvature of the roads and add to the road feature class
Copy the population density feature to gdb

In [3]:
# some data that needs to be processed
prjPath=r'C:\projects\ESRICrash\CrashAnalysisDOT\CobbCrashAnalysis\CobbCrashAnalysis.gdb'
fc= os.path.join(prjPath,'crash_3yr')
intersection =os.path.join(prjPath,'intersections')
AADT =os.path.join(prjPath,'traffic_counts')
traffic_flow=os.path.join(prjPath,'traffic_flow')
gdotAADT=os.path.join(prjPath,'AADT_2018_19yr_XY_Clip')
cobbRoads = os.path.join(prjPath,'CobbRoads')
pop_url='https://services.arcgis.com/P3ePLMYs2RVChkJx/arcgis/rest/services/USA_Census_BlockGroup_Areas_analysis_trim/FeatureServer/0'
arcpy.env.workspace=prjPath
arcpy.env.overwriteOutput=True


In [22]:
##_=arcpy.management.CopyFeatures(in_features=pop_url,out_feature_class='population')
##_=arcpy.management.AddFields('population',[['pop_density','Double']])
_=arcpy.management.CopyFeatures(in_features=pop_url,out_feature_class='population_wm')
_=arcpy.management.AddFields('population_wm',[['pop_density','Double']])

In [23]:
print ([f.name for f in arcpy.ListFields('population_wm')])

['OBJECTID', 'Shape', 'FIPS', 'COUNTY', 'STATE', 'POPULATION', 'SQMI', 'Shape_Length', 'Shape_Area', 'pop_density']


In [25]:
# select only Cobb County and create new feature class
popwm=arcpy.SelectLayerByAttribute_management('population_wm', 'NEW_SELECTION', 
                                        "COUNTY = 'Cobb'")
# Write the selected features to a new feature class
_=arcpy.management.CopyFeatures(popwm, 'cobbPopulation_wm')

In [27]:
##project to Georgia state plane west
out_coordinate_system = arcpy.SpatialReference(102667)
_=arcpy.Project_management('cobbPopulation_wm', 'cobbPopulation', out_coordinate_system)


In [35]:
# population density/per sq mile
density_cal=\
"""
def popDensity(pop, area):
   try:
      return pop*1.0/area
   except:
      return 0.0
"""
_=arcpy.management.CalculateField('cobbPopulation', 'pop_density', 'popDensity(!POPULATION!,!SQMI!)', 
                                  code_block=density_cal)

In [4]:
#spatial join  crash data with the population fc
field_mappings=arcpy.FieldMappings()
field_mappings.addTable(fc)
features_fields=arcpy.FieldMap()
features_fields.addInputField('cobbPopulation','pop_density')
field_mappings.addFieldMap(features_fields)
_=arcpy.analysis.SpatialJoin(fc,'cobbPopulation','crash_3yr_with_pop',join_type='KEEP_ALL',
                             match_option='intersects',field_mapping=field_mappings)

In [5]:

#add several new fields to the crash fc
fields=[['proximity_nearest_intersection','double']]  
_=arcpy.management.AddFields('crash_3yr_with_pop',fields)


In [6]:
#Cal proximity to intersection
_=arcpy.analysis.Near('crash_3yr_with_pop',intersection)
_=arcpy.management.CalculateField('crash_3yr_with_pop','proximity_nearest_intersection','!NEAR_DIST!')
_=arcpy.management.DeleteField('crash_3yr_with_pop',['NEAR_DIST','NEAR_FID'])

In [52]:
#prepare AADT Data, since Cobb AADT has lots of gaps. We can merge GDOT AADT with Cobb AADT data to improve data quality
# first to add a common field in GDOT AADT FC 
AADTfld=[['ADT','long']]
_=arcpy.management.AddFields(gdotAADT,AADTfld)
_=arcpy.management.CalculateField(gdotAADT,'ADT','!AADT_2019!')

In [53]:
#Merge GDOT AADT with Cobb AADT data to fill data gaps
_=arcpy.management.DeleteField(gdotAADT,['AADT_2019','AADT_2018','STATION_ID'])
_=arcpy.management.Merge([AADT,gdotAADT],'AADT_merged')

In [54]:
#Del duplicate fields
_=arcpy.management.DeleteField('AADT_merged',['X','Y'])

In [56]:
#curvature
curv=\
"""
import math
def curv(shp):
  x1=shp.firstPoint.x
  y1=shp.firstPoint.y
  x2=shp.lastPoint.x
  y2=shp.lastPoint.y
  
  eucliDistance=math.sqrt((x2-x1)**2+(y2-y1)**2)
  l=shp.length
  if eucliDistance>0:
     return l/eucliDistance
  return 1.0
"""

In [57]:
#add curvature field to the CobbRoads fc
curvfld=[['curv','double']]
_=arcpy.management.AddFields(cobbRoads,curvfld)
_=arcpy.management.CalculateField(cobbRoads,'curv','curv(!Shape!)', code_block=curv)

In [58]:
#spatial join AADT to the CobbRoads

_=arcpy.analysis.SpatialJoin(cobbRoads,'AADT_merged','CobbRoads_aadt',match_option='closest', search_radius='260 feet')

In [61]:
#spatial join  crash data with the traffic flow 
field_mappings=arcpy.FieldMappings()
field_mappings.addTable('CobbRoads_aadt')
features_fields=arcpy.FieldMap()
features_fields.addInputField(traffic_flow,'AADT')
field_mappings.addFieldMap(features_fields)
_=arcpy.analysis.SpatialJoin('CobbRoads_aadt',traffic_flow,'CobbRoads_AADT_flow',join_type='KEEP_ALL',
                             match_option='intersects',field_mapping=field_mappings,search_radius='10 feet')

In [74]:
code_block =\
"""
def getAADT(a,b):
   if not (a is None and b is None):
        return b
   elif (a is None and not b is None):
        return b
   elif (b is None and not a is None):
        return a
   else:
        pass
"""

In [63]:
#add one more field to get mean value of the aadt from gdot and cobb aadt data
aadtallfld=[['AADTall','long']]
_=arcpy.management.AddFields('CobbRoads_AADT_flow',aadtallfld)

In [75]:
_=arcpy.management.CalculateField('CobbRoads_AADT_flow','AADTall','getAADT(!AADT!,!ADT!)',code_block=code_block)

In [76]:
#add surface for each road within arcgis. This part will be computation heavy. Did this in desktop instead of python.
print ([f.name for f in arcpy.ListFields('CobbRoads_AADT_flow')])

['OBJECTID', 'Shape', 'Join_Count', 'TARGET_FID', 'Join_Count_1', 'TARGET_FID_1', 'TRANSSEGID', 'FUNCTIONALCLASS', 'NUMLANES', 'SPEED_LIMIT', 'LANE_TYPE', 'L_LADD', 'L_HADD', 'R_LADD', 'R_HADD', 'PREFIXDIR', 'ST_NAME', 'TYPE', 'SUFFIXDIR', 'QUAD', 'R_ZIP5', 'R_ZIP4', 'L_ZIP5', 'L_ZIP4', 'L_COMMUNIT', 'R_COMMUNIT', 'FROMTO_DIRECTION', 'TOFROM_DIRECTION', 'FROMEND_ELEV', 'TOENDELEV', 'HIERARCHY', 'DRIVETIME', 'ONEWAY', 'AM_ONEWAY', 'PM_ONEWAY', 'EDITOR', 'EDITDATE', 'DEPARTSOURCE', 'DATASOURCE', 'RECTIFIED', 'LIFECYCLE', 'FULLNAME', 'LENGTH', 'HWY_SHIELD', 'Jurisdiction', 'curv', 'ADT', 'AADT', 'AADTall', 'Avg_Slope', 'Shape_Length']


In [78]:
#delete some fields
del_fields=[
    'Join_Count','TARGET_FID','Join_Count_1','TARGET_FID_1','L_LADD','L_HADD','R_LADD','R_HADD', 
    'PREFIXDIR','ST_NAME','TYPE','SUFFIXDIR','QUAD','R_ZIP5','R_ZIP4','L_ZIP5','L_ZIP4', 
    'L_COMMUNIT','R_COMMUNIT','FROMTO_DIRECTION','TOFROM_DIRECTION','FROMEND_ELEV','TOENDELEV', 
    'HIERARCHY','ONEWAY','AM_ONEWAY','PM_ONEWAY','EDITOR','EDITDATE','DEPARTSOURCE', 
    'DATASOURCE','RECTIFIED','LIFECYCLE','HWY_SHIELD','Jurisdiction','ADT', 'AADT'
]

In [79]:
_=arcpy.management.DeleteField('CobbRoads_AADT_flow',del_fields)

In [80]:
print ([f.name for f in arcpy.ListFields('CobbRoads_AADT_flow')])

['OBJECTID', 'Shape', 'TRANSSEGID', 'FUNCTIONALCLASS', 'NUMLANES', 'SPEED_LIMIT', 'LANE_TYPE', 'ST_NAME', 'TYPE', 'DRIVETIME', 'FULLNAME', 'LENGTH', 'curv', 'AADTall', 'Avg_Slope', 'Shape_Length']


In [7]:
print ([f.name for f in arcpy.ListFields('crash_3yr_with_pop')])

['OBJECTID', 'Shape', 'Join_Count', 'TARGET_FID', 'ACCIDENTDATE', 'REPORTEDDATE', 'REPORTEDBY', 'UPDATED', 'UPDATEDBY', 'VEHICLES', 'INJURED', 'FATAL', 'SITE', 'ROADSURFACE', 'SEVERITY', 'WEATHER', 'TRAFFICFLOWTYPE', 'DESCRIPTION', 'CONTROLTYPE', 'LIGHTCONDITIONS', 'SPACING', 'QCLOCATION', 'QCREPORT', 'GEOCODED', 'ONROAD', 'INTERSECTION', 'ACCIDENTNUM', 'QCLOCATIONREASON', 'ICU', 'LOOPING', 'DATASOURCE', 'GEOCODEFLAG', 'NOTES', 'RMSID', 'IMP_ONROAD', 'IMP_INTERSECTION', 'DISTFROMINTERSECTION', 'DIRFROMINTERSECTION', 'V1_VEHICLETYPE', 'V2_VEHICLETYPE', 'V3_VEHICLETYPE', 'V4_VEHICLETYPE', 'V1_IMPACTPOINT', 'V2_IMPACTPOINT', 'V3_IMPACTPOINT', 'V4_IMPACTPOINT', 'V1_MOVEMENT', 'V2_MOVEMENT', 'V3_MOVEMENT', 'V4_MOVEMENT', 'V1_ESTSPEED', 'V2_ESTSPEED', 'V3_ESTSPEED', 'V4_ESTSPEED', 'V1_DRIVERCONDITION', 'V2_DRIVERCONDITION', 'V3_DRIVERCONDITION', 'V4_DRIVERCONDITION', 'V1_ALCOHOLTEST', 'V2_ALCOHOLTEST', 'V3_ALCOHOLTEST', 'V4_ALCOHOLTEST', 'V1_DRUGTEST', 'V2_DRUGTEST', 'V3_DRUGTEST', 'V4_DRUGT

In [8]:
#we need to delete some of null value fields and some driver condition fields 
delete_flds=['Join_Count','TARGET_FID','REPORTEDDATE','REPORTEDBY','UPDATED', 
             'UPDATEDBY','VEHICLES', 'INJURED', 'FATAL', 'SITE',
             'QCLOCATION','QCREPORT','GEOCODED', 'SPACING','TRANSSEGID',
             'ONROAD','DATASOURCE','GEOCODEFLAG','NOTES','RMSID','IMP_ONROAD',
             'IMP_INTERSECTION','DISTFROMINTERSECTION','DIRFROMINTERSECTION','V1_VEHICLETYPE','V2_VEHICLETYPE',
             'V3_VEHICLETYPE','V4_VEHICLETYPE', 'QCLOCATIONREASON','ICU','ACCIDENTNUM',
             'V1_IMPACTPOINT','V2_IMPACTPOINT','V3_IMPACTPOINT','V4_IMPACTPOINT', 
             'V2_MOVEMENT','V3_MOVEMENT','V4_MOVEMENT','V1_ESTSPEED', 
             'V2_ESTSPEED','V3_ESTSPEED','V4_ESTSPEED','V1_DRIVERCONDITION','ST_NAME', 
             'V2_DRIVERCONDITION','V3_DRIVERCONDITION','V4_DRIVERCONDITION', 
             'V1_ALCOHOLTEST','V2_ALCOHOLTEST','V3_ALCOHOLTEST','V4_ALCOHOLTEST', 
             'V1_DRUGTEST','V2_DRUGTEST','V3_DRUGTEST','V4_DRUGTEST','V1_PEDMOVEMENT', 
             'V2_PEDMOVEMENT','V3_PEDMOVEMENT','V4_PEDMOVEMENT','V1_DIRECTION', 
             'V2_DIRECTION','V3_DIRECTION','V4_DIRECTION','V2_FACTOR1', 'LOOPING',
             'V3_FACTOR1','V4_FACTOR1','V1_FACTOR2','V1_FACTOR3','V1_FACTOR4', 
             'V2_FACTOR2','V2_FACTOR3','V2_FACTOR4', 'V3_FACTOR2', 'V3_FACTOR3', 
             'V3_FACTOR4', 'V4_FACTOR2', 'V4_FACTOR3', 'V4_FACTOR4', 'V1_E2', 
             'V2_E2', 'V3_E1', 'V3_E2', 'V4_E1', 'V4_E2', 'V1_E1_MOSTHARMFUL', 
             'V1_E2_MOSTHARMFUL', 'V2_E1_MOSTHARMFUL', 'V2_E2_MOSTHARMFUL', 'V3_E1_MOSTHARMFUL', 
             'V3_E2_MOSTHARMFUL', 'V4_E1_MOSTHARMFUL', 'V4_E2_MOSTHARMFUL', 'DOT_E1', 'DOT_E2',
             'EDITOR', 'EDITDATE', 'OFFICEQC', 'FATALINJUREQC', 'GEOCODERSERVER', 'REMOVED',
             ]

In [9]:
_=arcpy.management.DeleteField('crash_3yr_with_pop',delete_flds)

In [15]:
print ([f.name for f in arcpy.ListFields('crash_3yr_with_pop')])

['OBJECTID', 'Shape', 'ACCIDENTDATE', 'ROADSURFACE', 'SEVERITY', 'WEATHER', 'TRAFFICFLOWTYPE', 'DESCRIPTION', 'CONTROLTYPE', 'LIGHTCONDITIONS', 'INTERSECTION', 'V1_MOVEMENT', 'V1_FACTOR1', 'V1_E1', 'V2_E1', 'pop_density', 'proximity_nearest_intersection']


In [11]:
#spatial join AADT to the CobbRoads
_=arcpy.analysis.SpatialJoin('crash_3yr_with_pop','CobbRoads_AADT_flow','Crash_joined_3yr',
                             join_type='KEEP_COMMON', match_option='closest', search_radius='100 feet')

In [12]:
print ([f.name for f in arcpy.ListFields('Crash_joined_3yr')])

['OBJECTID', 'Shape', 'Join_Count', 'TARGET_FID', 'ACCIDENTDATE', 'ROADSURFACE', 'SEVERITY', 'WEATHER', 'TRAFFICFLOWTYPE', 'DESCRIPTION', 'CONTROLTYPE', 'LIGHTCONDITIONS', 'INTERSECTION', 'V1_MOVEMENT', 'V1_FACTOR1', 'V1_E1', 'V2_E1', 'pop_density', 'proximity_nearest_intersection', 'TRANSSEGID', 'FUNCTIONALCLASS', 'NUMLANES', 'SPEED_LIMIT', 'LANE_TYPE', 'ST_NAME', 'TYPE', 'DRIVETIME', 'FULLNAME', 'LENGTH', 'curv', 'AADTall', 'Avg_Slope']


In [46]:
_=arcpy.management.DeleteField('Crash_joined_3yr',['Join_Count','TRAFFICFLOWTYPE','INTERSECTION','TARGET_FID','V2_E1','TRANSSEGID','AADTall','ST_NAME','TYPE','DRIVETIME','FULLNAME'])

In [4]:
import arcgis
df=arcgis.features.GeoAccessor.from_featureclass('crash_joined_3yr')

In [5]:
df.head()

,OBJECTID,ACCIDENTDATE,ROADSURFACE,SEVERITY,WEATHER,DESCRIPTION,CONTROLTYPE,LIGHTCONDITIONS,V1_MOVEMENT,V1_FACTOR1,...,pop_density,proximity_nearest_intersection,FUNCTIONALCLASS,NUMLANES,SPEED_LIMIT,LANE_TYPE,LENGTH,curv,Avg_Slope,SHAPE
0,1,2018-03-18 23:10:00.000001,2.0,0.0,3.0,4.0,7.0,4.0,6.0,11.0,...,2136.585366,0.000000,MAJOR,4,25.0,1,496.87168,1.010868,9.374528,"{""x"": 2200251.299880162, ""y"": 1422066.99999782..."
1,2,2018-03-18 22:31:00.000000,1.0,0.0,1.0,6.0,7.0,5.0,5.0,10.0,...,2178.181818,0.000000,MAJOR,2,35.0,1,140.24278,1.001089,7.413122,"{""x"": 2173367.799841497, ""y"": 1399011.09991548..."
2,3,2018-03-18 21:56:00.000001,1.0,4.0,2.0,5.0,2.0,4.0,3.0,4.0,...,3075.581395,0.000000,MAJOR,2,35.0,1,44.20401,1.000000,31.375962,"{""x"": 2166057.800025828, ""y"": 1405272.50012232..."
3,4,2018-03-18 17:02:00.000000,1.0,4.0,1.0,3.0,7.0,1.0,5.0,3.0,...,2075.000000,340.081911,INTERSTATE,-999,65.0,2,2688.72923,1.005217,8.160925,"{""x"": 2197852.34962558, ""y"": 1390927.089508742..."
4,5,2018-03-18 17:27:00.000000,1.0,0.0,1.0,3.0,7.0,1.0,5.0,3.0,...,2075.000000,442.469009,INTERSTATE,-999,65.0,2,2688.72923,1.005217,8.160925,"{""x"": 2198188.9316295795, ""y"": 1391296.8066170..."


In [6]:
df.isnull().sum()

OBJECTID                             0
ACCIDENTDATE                         0
ROADSURFACE                        809
SEVERITY                          1949
WEATHER                           8624
DESCRIPTION                        195
CONTROLTYPE                       1515
LIGHTCONDITIONS                    811
V1_MOVEMENT                       1900
V1_FACTOR1                        3754
V1_E1                             1415
pop_density                        208
proximity_nearest_intersection       0
FUNCTIONALCLASS                     39
NUMLANES                             0
SPEED_LIMIT                         18
LANE_TYPE                            0
LENGTH                               0
curv                                 0
Avg_Slope                            1
SHAPE                                0
dtype: int64

In [7]:
df.shape

(74431, 21)

In [8]:
df['SPEED_LIMIT'].fillna(df['SPEED_LIMIT'].median(), inplace=True)

In [9]:
df2 = df.dropna()

In [10]:
df2.shape

(61499, 21)

In [11]:
df2.isnull().sum()

OBJECTID                          0
ACCIDENTDATE                      0
ROADSURFACE                       0
SEVERITY                          0
WEATHER                           0
DESCRIPTION                       0
CONTROLTYPE                       0
LIGHTCONDITIONS                   0
V1_MOVEMENT                       0
V1_FACTOR1                        0
V1_E1                             0
pop_density                       0
proximity_nearest_intersection    0
FUNCTIONALCLASS                   0
NUMLANES                          0
SPEED_LIMIT                       0
LANE_TYPE                         0
LENGTH                            0
curv                              0
Avg_Slope                         0
SHAPE                             0
dtype: int64

In [13]:
df2['RIGHTANGLE']=1.0
df2.loc[df2['DESCRIPTION']>1,'RIGHTANGLE']=0.0

C:\Users\zhangc\AppData\Local\ESRI\conda\envs\sparkesri\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\zhangc\AppData\Local\ESRI\conda\envs\sparkesri\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [14]:
df2.loc[df['NUMLANES']==-999,'NUMLANES']=6 #interstate 

In [15]:
#this idea came from Daniel's github notebook althogh the new version is much different.
l = LocationInfo("Atlanta", "USA", "US/Eastern", 33.7490, -84.3880)

l.solar_depression = 'civil'

def solar_azimuth(x):
    try:
        return astral.sun.azimuth(l.observer,x)
    except:
        np.nan
def solar_elevation(x):
    try:
        return astral.sun.elevation(l.observer,x)
    except:
        return np.nan


In [17]:
df2['solar_azimuth']=pd.DatetimeIndex(df2.ACCIDENTDATE).map(solar_azimuth)
df2['solar_elevation']=pd.DatetimeIndex(df2.ACCIDENTDATE).map(solar_elevation)

C:\Users\zhangc\AppData\Local\ESRI\conda\envs\sparkesri\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\zhangc\AppData\Local\ESRI\conda\envs\sparkesri\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [18]:
df2.head()

,OBJECTID,ACCIDENTDATE,ROADSURFACE,SEVERITY,WEATHER,DESCRIPTION,CONTROLTYPE,LIGHTCONDITIONS,V1_MOVEMENT,V1_FACTOR1,...,NUMLANES,SPEED_LIMIT,LANE_TYPE,LENGTH,curv,Avg_Slope,SHAPE,RIGHTANGLE,solar_azimuth,solar_elevation
0,1,2018-03-18 23:10:00.000001,2.0,0.0,3.0,4.0,7.0,4.0,6.0,11.0,...,4,25.0,1,496.87168,1.010868,9.374528,"{""x"": 2200251.299880162, ""y"": 1422066.99999782...",0.0,264.483218,7.105017
1,2,2018-03-18 22:31:00.000000,1.0,0.0,1.0,6.0,7.0,5.0,5.0,10.0,...,2,35.0,1,140.24278,1.001089,7.413122,"{""x"": 2173367.799841497, ""y"": 1399011.09991548...",0.0,258.818460,15.056053
2,3,2018-03-18 21:56:00.000001,1.0,4.0,2.0,5.0,2.0,4.0,3.0,4.0,...,2,35.0,1,44.20401,1.000000,31.375962,"{""x"": 2166057.800025828, ""y"": 1405272.50012232...",0.0,253.354698,22.092605
3,4,2018-03-18 17:02:00.000000,1.0,4.0,1.0,3.0,7.0,1.0,5.0,3.0,...,6,65.0,2,2688.72923,1.005217,8.160925,"{""x"": 2197852.34962558, ""y"": 1390927.089508742...",0.0,161.268741,54.001393
4,5,2018-03-18 17:27:00.000000,1.0,0.0,1.0,3.0,7.0,1.0,5.0,3.0,...,6,65.0,2,2688.72923,1.005217,8.160925,"{""x"": 2198188.9316295795, ""y"": 1391296.8066170...",0.0,171.863581,55.218132


In [19]:
df2.to_csv('trainingData/rightAnglecrash.csv')

In [57]:
df2.spatial.to_featureclass(location=os.path.join(prjPath,'rightangleRFS'), overwrite=True)

C:\Users\zhangc\AppData\Local\ESRI\conda\envs\sparkesri\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


'C:\\projects\\ESRICrash\\CrashAnalysisDOT\\CobbCrashAnalysis\\CobbCrashAnalysis.gdb\\rightangleRFS'

2.4.1
